In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import cv2
from matplotlib import pyplot as plt
import os
import shutil
import numpy as np
from PIL import Image
import torch
from sklearn.model_selection import train_test_split
import random

# tensorflow library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import metrics

In [7]:
current_dir = os.getcwd()
print("Current Working Directory:", current_dir)
new_directory = '/content/drive/My Drive/IT1244/experimental'
os.chdir(new_directory)

# Verify the change
print("New Current Working Directory:", os.getcwd())

Current Working Directory: /content
New Current Working Directory: /content/drive/My Drive/IT1244/experimental


In [8]:
classes=set()

if not os.listdir("data_classes"):
    for image in os.listdir("../images"):
        classes.add(image[:2])

    #the following code extracts the numerical part of the class name
    #this is because some of the numeric values are 1 digit and will leave an underscore behind in the string
    #this would filter out the numeric portion
    newlist=[int(''.join(filter(str.isdigit, classname))) for classname in classes]


    parent_dir="../experimental/data_classes"
    imagePath="../images"

    for num in (newlist):
        dir_path = os.path.join(parent_dir, str(num))
        os.makedirs(dir_path, exist_ok=True)
        print(f"Directory {dir_path} created.")


    for imagefilename in os.listdir(imagePath):
        newstring = "".join(filter(str.isdigit, imagefilename[:2]))
        targetdirectory=os.path.join(parent_dir, newstring)
        newimagepath=os.path.join(imagePath, imagefilename)
        shutil.copy(newimagepath, targetdirectory)
        print("image moved")

In [9]:
add_datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    zoom_range=0.2,     # Randomly zoom in/out by up to 20%
    brightness_range=[0.8, 1.2]
)

base_dir = '../experimental/data_classes'
#choose 514 because the max number of images among the classes is 514
target_count = 400
for class_num in range(23):
    class_dir = os.path.join(base_dir, str(class_num))
    images = os.listdir(class_dir)
    current_count = len(images)

    if current_count < target_count:
        print(f"Augmenting class {class_num} with {target_count - current_count} new images.")

        #this is because we want to top out the number of images in each class folders from 0 to  22
        images_needed = target_count - current_count

        # Augment images
        for img_name in images:
            img_path = os.path.join(class_dir, img_name)
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # Generate new images
            generated_images = 0  # Track images generated for this specific image
            for batch in add_datagen.flow(x, batch_size=1, save_to_dir=class_dir, save_format='png'):
                generated_images += 1
                current_count += 1  # Update the current image count for the class

                # Stop generating if the target count is reached
                if current_count >= target_count:
                    break

            # If the target count is reached for this class, stop further augmentation
            if current_count >= target_count:
                break

    elif current_count > target_count:
        # Remove excess images to reach the target count
        images_to_remove = current_count - target_count
        print(f"Reducing class {class_num} by removing {images_to_remove} images.")

        # Randomly select images to remove
        images_to_delete = random.sample(images, images_to_remove)
        for img_name in images_to_delete:
            img_path = os.path.join(class_dir, img_name)
            os.remove(img_path)


Reducing class 0 by removing 114 images.
Reducing class 1 by removing 100 images.
Augmenting class 2 with 56 new images.
Augmenting class 3 with 132 new images.
Augmenting class 4 with 140 new images.
Augmenting class 5 with 156 new images.
Augmenting class 6 with 180 new images.
Augmenting class 7 with 182 new images.
Augmenting class 8 with 186 new images.
Augmenting class 9 with 198 new images.
Augmenting class 10 with 198 new images.
Augmenting class 11 with 202 new images.
Augmenting class 12 with 216 new images.
Augmenting class 13 with 244 new images.
Augmenting class 14 with 250 new images.
Augmenting class 15 with 260 new images.
Augmenting class 16 with 260 new images.
Augmenting class 17 with 268 new images.
Augmenting class 18 with 270 new images.
Augmenting class 19 with 272 new images.
Augmenting class 20 with 274 new images.
Augmenting class 21 with 282 new images.
Augmenting class 22 with 292 new images.


In [10]:
# Verify the number of images in each class directory
for class_num in range(23):
    class_dir = os.path.join(base_dir, str(class_num))
    images = os.listdir(class_dir)
    print(f"Class {class_num} now has {len(images)} images.")

Class 0 now has 400 images.
Class 1 now has 400 images.
Class 2 now has 400 images.
Class 3 now has 400 images.
Class 4 now has 400 images.
Class 5 now has 397 images.
Class 6 now has 399 images.
Class 7 now has 396 images.
Class 8 now has 397 images.
Class 9 now has 399 images.
Class 10 now has 398 images.
Class 11 now has 396 images.
Class 12 now has 399 images.
Class 13 now has 396 images.
Class 14 now has 396 images.
Class 15 now has 398 images.
Class 16 now has 396 images.
Class 17 now has 397 images.
Class 18 now has 394 images.
Class 19 now has 398 images.
Class 20 now has 398 images.
Class 21 now has 399 images.
Class 22 now has 394 images.


In [11]:
# we want to split the data into train, val, test
# to do so we split the data such that 70% of the data are in training, 20% are in validation and 10% are in testing
# we spilt the image data into new folders as shown below
original_dir = '../experimental/data_classes'
train_dir = '../experimental/new/train'
val_dir = '../experimental/new/validation'
test_dir = '../experimental/new/test'

for i in range(23):
    os.makedirs(os.path.join(train_dir, str(i)), exist_ok=True)
    os.makedirs(os.path.join(val_dir, str(i)), exist_ok=True)
    os.makedirs(os.path.join(test_dir, str(i)), exist_ok=True)

for i in range(23):
    class_path = os.path.join(original_dir, str(i))
    images = os.listdir(class_path)
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=1/3, random_state=42)

# I think that copy files might be more time consuming than moving
# But I chose to copy instead of move because mistakes can be made
# hence, when mistakes are made to the train, validation and test folders, I still have the images in teh data_classes that I can copy over to these 3 folders
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, str(i)))
    for img in val_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, str(i)))
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, str(i)))


In [12]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=256,
    class_mode='sparse',
    shuffle = True
)

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128,128),
    batch_size=256,
    class_mode='sparse',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=256,
    class_mode='sparse',
    shuffle=True
)

Found 6394 images belonging to 23 classes.
Found 1833 images belonging to 23 classes.
Found 920 images belonging to 23 classes.


In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [58]:
import numpy as np
import tensorflow as tf

def testing_model(model, test_generator, class_of_interest=6):
    y_true = []
    y_pred = []

    # Calculate the number of batches in the test set
    num_batches = len(test_generator)

    for _ in range(num_batches):
        images, labels = next(test_generator)
        predictions = model.predict(images)
        y_pred.extend(np.argmax(predictions, axis=1))

        # Check if labels are tensors and convert to numpy arrays if necessary
        if isinstance(labels, tf.Tensor):
            y_true.extend(labels.numpy())
        else:
            y_true.extend(labels)

    y_true, y_pred = np.array(y_true), np.array(y_pred)

    false_negatives = np.sum((y_true == class_of_interest) & (y_pred != class_of_interest))
    total_positives = np.sum(y_true == class_of_interest)

    false_negative_rate = false_negatives / total_positives if total_positives > 0 else 0

    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"False Negative Rate for class {class_of_interest}: {false_negative_rate:.4f}")


In [15]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.GaussianNoise(0.1)
])

In [29]:
model = Sequential([
    Input(shape=(128, 128, 3)),
    data_augmentation,
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.4),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(23, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', 'precision', 'recall'])

model.summary()

logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 63, 63, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 63, 63, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 31, 31, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 31, 31, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 31, 31, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 31, 31, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 28800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 512)                 │      14,746,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 23)                  │          11,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,854,103 (56.66 MB)

 Trainable params: 14,852,631 (56.66 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [30]:
train_generator.class_indices

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22}

In [41]:
val_generator.class_indices

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22}

In [31]:
hist = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    verbose=1
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.0427 - loss: 4.4436 - val_accuracy: 0.0436 - val_loss: 3.4596
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.0561 - loss: 3.8045 - val_accuracy: 0.0431 - val_loss: 3.7296
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.0883 - loss: 3.4374 - val_accuracy: 0.0447 - val_loss: 3.7271
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.1194 - loss: 3.2926 - val_accuracy: 0.1167 - val_loss: 4.0159
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.1759 - loss: 2.9762 - val_accuracy: 0.0922 - val_loss: 3.8986
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.2261 - loss: 2.7364 - val_accuracy: 0.0895 - val_loss: 3.8005
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.2489 - loss: 2.6329 - val_accuracy: 0.0851 - val_loss: 3.7096
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.2686 - loss: 2.5501 - val_accuracy: 0.0753 - val_loss:

In [18]:
import keras
import torch
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import MobileNetV2

base_model2 = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(128,128, 3)
)

base_model2.trainable = False  # No training on this model
inputs1 = keras.Input(shape=(128,128, 3))
x2 = data_augmentation(inputs1)
x2 = base_model2(inputs1, training=False)
x2 = keras.layers.GlobalAveragePooling2D()(x2)  # Pooling layer to reduce dimensions
outputs1 = keras.layers.Dense(23, activation="softmax")(x2)  # Output layer for 23 classes

model2 = keras.Model(inputs1, outputs1)
model2.summary(show_trainable=True)

# Compile the model (no training, just for evaluation)
model2.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", "precision", "recall"]
)

# Train only the output layer
print("Training the last layer of Model 1 on the training data:")
model2.fit(train_generator, epochs=10, validation_data=val_generator)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_3 (InputLayer)          │ (None, 128, 128, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ mobilenetv2_1.00_128 (Functional)   │ (None, 4, 4, 1280)           │     2,257,984 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d            │ (None, 1280)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense_2 (Dense)                     │ (None, 23)                   │        29,463 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 2,287,447 (8.73 MB)

 Trainable params: 29,463 (115.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Training the last layer of Model 1 on the training data:
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.3341 - loss: 2.4409 - val_accuracy: 0.8200 - val_loss: 0.6908
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.8699 - loss: 0.5181 - val_accuracy: 0.8974 - val_loss: 0.3991
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9280 - loss: 0.3075 - val_accuracy: 0.9302 - val_loss: 0.2871
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9606 - loss: 0.1995 - val_accuracy: 0.9411 - val_loss: 0.2291
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9711 - loss: 0.1563 - val_accuracy: 0.9531 - val_loss: 0.1902
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9795 - loss: 0.1260 - val_accuracy: 0.9618 - val_loss: 0.1624
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.9854 - loss: 0.1002 - val_accuracy: 0.9694 - val_loss: 0.1403
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy:

In [59]:
testing_model(model2, test_generator, "14")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9755 - loss: 0.1082
Test Loss: 0.1062
Test Accuracy: 0.9772
False Negative Rate for class 14: 0.0000


In [40]:
# Model 3: mobilenetv2 with finetuning
base_model3 = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3)
)

base_model3.trainable = False  # Initially freeze the base model

inputs3 = keras.Input(shape=(128, 128, 3))
x3 = base_model3(inputs3, training=False)
x3 = keras.layers.GlobalAveragePooling2D()(x3)  # Pooling layer to reduce dimensions
outputs3 = keras.layers.Dense(23, activation="softmax")(x3)

model3 = keras.Model(inputs3, outputs3)

# Optionally unfreeze some layers in mobilenetv2 for fine-tuning
for layer in base_model3.layers[-10:]:  # Unfreeze the last 10 layers
    layer.trainable = True

model3.summary(show_trainable = True)

# Re-compile the model with a lower learning rate for fine-tuning
model3.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy"
    ]
)

# Fine-tune Model 3 on the training data
print("Fine-tuning MobileNetv2...")
model3.fit(train_generator, epochs=10, validation_data=val_generator)


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_24 (InputLayer)         │ (None, 128, 128, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ mobilenetv2_1.00_128 (Functional)   │ (None, 4, 4, 1280)           │     2,257,984 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d_8          │ (None, 1280)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense_17 (Dense)                    │ (None, 23)                   │        29,463 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 2,287,447 (8.73 MB)

 Trainable params: 761,943 (2.91 MB)

 Non-trainable params: 1,525,504 (5.82 MB)

Fine-tuning MobileNetv2...
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.7568 - loss: 0.9398 - val_accuracy: 0.6759 - val_loss: 1.3796
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.9970 - loss: 0.0210 - val_accuracy: 0.6274 - val_loss: 1.7049
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 0.6088 - val_loss: 2.0722
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.9994 - loss: 0.0021 - val_accuracy: 0.6219 - val_loss: 1.9147
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.6498 - val_loss: 1.6746
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 1.0000 - loss: 7.4421e-04 - val_accuracy: 0.6716 - val_loss: 1.4558
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 1.0000 - loss: 6.1549e-04 - val_accuracy: 0.7016 - val_loss: 1.2606
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 1.0000 - loss: 4.6502

In [63]:
testing_model(model3, test_generator, "14")

5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 804ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8229 - loss: 0.7730
Test Loss: 0.7752
Test Accuracy: 0.8261
False Negative Rate for class 14: 0.0000


In [48]:
from tensorflow.keras.applications import ResNet50

resnet_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(128,128,3),
)

resnet_model.trainable = False
inputs4 = keras.Input(shape = (128,128,3))
x4 = resnet_model(inputs4, training = False)
x4 = GlobalAveragePooling2D()(x4)
x4 = Dropout(0.2)(x4)
outputs4 = Dense(23, activation = "softmax")(x4)
model4 = keras.Model(inputs4, outputs4)
model4.summary(show_trainable=True)
model4.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy"
    ]
)

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)                        ┃ Output Shape                 ┃       Param # ┃ Traina… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ input_layer_34 (InputLayer)         │ (None, 128, 128, 3)          │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ resnet50 (Functional)               │ (None, 4, 4, 2048)           │    23,587,712 │    N    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ global_average_pooling2d_13         │ (None, 2048)                 │             0 │    -    │
│ (GlobalAveragePooling2D)            │                              │               │         │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dropout_19 (Dropout)                │ (None, 2048)                 │             0 │    -    │
├─────────────────────────────────────┼──────────────────────────────┼───────────────┼─────────┤
│ dense_22 (Dense)                    │ (None, 23)                   │        47,127 │    Y    │
└─────────────────────────────────────┴──────────────────────────────┴───────────────┴─────────┘

 Total params: 23,634,839 (90.16 MB)

 Trainable params: 47,127 (184.09 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [50]:
model4.fit(train_generator, epochs = 10, validation_data = val_generator)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.0495 - loss: 3.3419 - val_accuracy: 0.1102 - val_loss: 3.0648
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.0923 - loss: 3.0696 - val_accuracy: 0.2722 - val_loss: 2.9808
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.1605 - loss: 2.9744 - val_accuracy: 0.3939 - val_loss: 2.9084
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.2093 - loss: 2.9054 - val_accuracy: 0.3999 - val_loss: 2.8453
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.2803 - loss: 2.8225 - val_accuracy: 0.4463 - val_loss: 2.7870
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.3129 - loss: 2.7651 - val_accuracy: 0.4774 - val_loss: 2.7322
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.3681 - loss: 2.7104 - val_accuracy: 0.4997 - val_loss: 2.6829
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.3910 - loss: 2.6585 - val_accuracy: 0.5510 - val_loss:

In [60]:
testing_model(model4, test_generator, "14")

5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5376 - loss: 2.5361
Test Loss: 2.5385
Test Accuracy: 0.5337
False Negative Rate for class 14: 0.0000
